In [ ]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19880 sha256=4892352910c099a5d2be7f45982ea6587297e85ace4de65298266e0cb451e157
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/Data Mining/Assignment 3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/Data Mining/Assignment 3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Mounted at /gdrive
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


In [ ]:
TrainCols = list(train_data.columns.values)
TestCols = list(test_data.columns.values)
print(TrainCols)
print(TestCols)

['CoverageField11A', 'CoverageField11B', 'CoverageField1A', 'CoverageField1B', 'CoverageField2A', 'CoverageField2B', 'CoverageField3A', 'CoverageField3B', 'CoverageField4A', 'CoverageField4B', 'CoverageField5A', 'CoverageField5B', 'CoverageField6A', 'CoverageField6B', 'Field10', 'Field11', 'Field7', 'Field8', 'Field9', 'GeographicField10A', 'GeographicField10B', 'GeographicField11A', 'GeographicField11B', 'GeographicField12A', 'GeographicField12B', 'GeographicField13A', 'GeographicField13B', 'GeographicField14A', 'GeographicField14B', 'GeographicField15A', 'GeographicField15B', 'GeographicField16A', 'GeographicField16B', 'GeographicField17A', 'GeographicField17B', 'GeographicField18A', 'GeographicField18B', 'GeographicField19A', 'GeographicField19B', 'GeographicField1A', 'GeographicField1B', 'GeographicField20A', 'GeographicField20B', 'GeographicField21A', 'GeographicField21B', 'GeographicField22A', 'GeographicField22B', 'GeographicField23A', 'GeographicField23B', 'GeographicField24A',

In [ ]:
# Seperate Target column from Train Data
Xtrain = train_data[TrainCols[0:len(TrainCols)-1]].copy()
Ytrain = train_data[['QuoteConversion_Flag']].copy()
print("Train Set shape:")
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = test_data.copy()
print("Test Set shape:")
print(Xtest.shape)

Train Set shape:
(65000, 595)
(65000, 1)
Test Set shape:
(173836, 596)


In [ ]:
print(Xtest.columns)

Index(['CoverageField11A', 'CoverageField11B', 'CoverageField1A',
       'CoverageField1B', 'CoverageField2A', 'CoverageField2B',
       'CoverageField3A', 'CoverageField3B', 'CoverageField4A',
       'CoverageField4B',
       ...
       'PropertyField38_N', 'PropertyField38_Y', 'GeographicField63_ ',
       'GeographicField63_N', 'GeographicField63_Y', 'GeographicField64_CA',
       'GeographicField64_IL', 'GeographicField64_NJ', 'GeographicField64_TX',
       'GeographicField64'],
      dtype='object', length=596)


In [ ]:
Xtest.drop(["GeographicField64"],axis=1,inplace=True)


In [ ]:
sum(pd.isna(Ytrain).sum())

0

In [ ]:
Ytrain.replace('', np.NaN)

,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0
...,...
64995,0
64996,0
64997,0
64998,0


MLP

In [ ]:
#Construct Multilayer Perceptron using the best parameters
#Default mode
mlp = MLPClassifier(random_state=1, max_iter=3)
mlp.fit(Xtrain, Ytrain)
mlp_predict=mlp.predict(Xtest)

Export for Kaggle

In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(mlp.predict(Xtest),columns=["Prediction"])
print(mlp.predict(Xtest))
pred.head()

[0 0 0 ... 0 0 0]


,Prediction
0,0
1,0
2,0
3,0
4,0


In [ ]:
#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(mlp.predict_proba(Xtest))

pred_Probability.head()

,0,1
0,1.0,4.271044e-44
1,1.0,3.590381e-16
2,1.0,5.198435e-37
3,1.0,6.448453e-36
4,1.0,1.762531e-34


In [ ]:
pred.to_csv('/gdrive/My Drive/Data Mining/Assignment 3/MLPOnlyResults2.csv')